In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 0
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)

In [4]:
sizes = [0.5]
for size in sizes:
    train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_basic_{size}_seed_{SEED}.csv')
    train_df = train_df.fillna(-1)
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)
    for steps in [int(15e6)]:
        dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
            f'../../../anemia_ml4hc/models/sb/new_different_training_sizes/dqn_pr_train_set_{size}_{SEED}_{steps}')

using just stable baselines (not 3)
--------------------------------------
| % time spent exploring  | 91       |
| episodes                | 50000    |
| mean 100 episode reward | -0.7     |
| steps                   | 135700   |
| success rate            | 0.19     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 82       |
| episodes                | 100000   |
| mean 100 episode reward | -0.7     |
| steps                   | 280877   |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 72       |
| episodes                | 150000   |
| mean 100 episode reward | -0.9     |
| steps                   | 435023   |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 62       |
| episodes                | 200000   |
| mean 100 episode reward | 

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1550000  |
| mean 100 episode reward | 0.1      |
| steps                   | 5223754  |
| success rate            | 0.6      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1600000  |
| mean 100 episode reward | 0.1      |
| steps                   | 5412047  |
| success rate            | 0.6      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1650000  |
| mean 100 episode reward | 0.2      |
| steps                   | 5598218  |
| success rate            | 0.6      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1700000  |
| mean 100 episode reward | 0.3      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3100000  |
| mean 100 episode reward | 0.6      |
| steps                   | 11504859 |
| success rate            | 0.85     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3150000  |
| mean 100 episode reward | 0.6      |
| steps                   | 11715701 |
| success rate            | 0.88     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3200000  |
| mean 100 episode reward | 0.7      |
| steps                   | 11925694 |
| success rate            | 0.89     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3250000  |
| mean 100 episode reward | 0.7      |
| steps                  

In [ ]:
# noise_levels = [0.1, 0.2, 0.3]

# for level in noise_levels:
# # for level in noise_levels[:1]:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_noisiness_{level}.csv')
# #     print(len(train_df))
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     print(X_train.shape, y_train.shape)
    
#     for steps in [int(15e6), int(12e6), int(10e6)]:
# #     for steps in [int(2e3)]:
#         dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
#                     f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/dqn_pr_noisiness_{level}_{steps}')#,

#### The Data

In [ ]:
# train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = pd.read_csv('../../../anemia_ml4hc/data/train_set_basic.csv')
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = train_df.fillna(-1)
train_df.head()

In [ ]:
len(train_df)

In [ ]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

In [ ]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### Training 

In [ ]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
# for steps in [int(50e6)]:
#     dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/complete_models/dueling_ddqn_{steps}',
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dddqn_basic')
    
for steps in [int(15e6), int(10e6), int(5e6)]:
# for steps in [int(2e3)]:
    dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
                                 f'../../../anemia_ml4hc/models/sb/prioritized_replay/dqn_{steps}')#,
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dqn_basic')

#### Testing

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)